In [1]:
## Instalando Pandera
!pip install pandera

     |████████████████████████████████| 162 kB 14.6 MB/s 
     |████████████████████████████████| 234 kB 23.2 MB/s 
     |████████████████████████████████| 245 kB 21.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 45 kB 3.6 MB/s 
     |████████████████████████████████| 636 kB 24.9 MB/s 
  Created wheel for petl: filename=petl-1.7.4-py3-none-any.whl size=217001 sha256=84f8a27eeddef32d4fa5f995b508740c5e95c16f43bf0fe82eff97e531f7cf0d
  Stored in directory: /root/.cache/pip/wheels/75/23/fb/a0e8e350a8718579b0cde55163df77b94de835d7d116388fa5
  Created wheel for simpleeval: filename=simpleeval-0.9.10-py3-none-any.whl size=13769 sha256=d0bda2983122fc5503ca92e6d43ecabebacb6ce03a54a269366a2d10cc8ae927
  Stored in directory: /root/.cache/pip/wheels/d4/aa/50/0b420d1eabad3c16a82368935c6a2050955bc3ee2a11ee4e06
  Created wheel for stringcase: filename=stringcase-1.2.0-py3-none

In [2]:
import pandas as pd
import pandera as pa

In [3]:
valores_ausentes = ['**', '***', '###!', '####', '****', '*****', 'NULL']
df = pd.read_csv("ocorrencia_2010_2020.csv", parse_dates=['ocorrencia_dia'], dayfirst=True, na_values=valores_ausentes)
df.head(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
5,39807,39807,INCIDENTE,SALVADOR,BA,NaN,2010-01-06,17:53:00,0
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0


In [4]:
# Criando schema para validar dados
schema = pa.DataFrameSchema(
    columns = {
        "codigo_ocorrencia": pa.Column(pa.Int),
        "codigo_ocorrencia2": pa.Column(pa.Int),
        "ocorrencia_classificacao": pa.Column(pa.String),
        "ocorrencia_cidade": pa.Column(pa.String),
        "ocorrencia_uf": pa.Column(pa.String, pa.Check.str_length(2,3), nullable=True),
        "ocorrencia_aerodromo": pa.Column(pa.String, nullable=True),
        "ocorrencia_dia": pa.Column(pa.DateTime),
        "ocorrencia_hora": pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9]):([0-5][0-9])?$') ,nullable=True),
        "total_recomendacoes": pa.Column(pa.Int)
    }
)

In [5]:
# Validando dados de acordo com schema
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,NaN,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,NaN,2020-12-31,09:00:00,0


In [6]:
df.loc[1]

codigo_ocorrencia                         40349
codigo_ocorrencia2                        40349
ocorrencia_classificacao              INCIDENTE
ocorrencia_cidade                         BELÉM
ocorrencia_uf                                PA
ocorrencia_aerodromo                       SBBE
ocorrencia_dia              2010-01-03 00:00:00
ocorrencia_hora                        11:05:00
total_recomendacoes                           0
Name: 1, dtype: object

In [7]:
df.iloc[-1]

codigo_ocorrencia                         79844
codigo_ocorrencia2                        79844
ocorrencia_classificacao              INCIDENTE
ocorrencia_cidade                RIO DE JANEIRO
ocorrencia_uf                                RJ
ocorrencia_aerodromo                       SBJR
ocorrencia_dia              2020-12-31 00:00:00
ocorrencia_hora                        13:24:00
total_recomendacoes                           0
Name: 5751, dtype: object

In [8]:
df.loc[:, 'ocorrencia_uf']

0       RJ
1       PA
2       RJ
3       MT
4       RS
        ..
5747    SP
5748    TO
5749    RJ
5750    MS
5751    RJ
Name: ocorrencia_uf, Length: 5752, dtype: object

In [9]:
df['ocorrencia_uf']

0       RJ
1       PA
2       RJ
3       MT
4       RS
        ..
5747    SP
5748    TO
5749    RJ
5750    MS
5751    RJ
Name: ocorrencia_uf, Length: 5752, dtype: object

In [10]:
# Exibindo total de valores nulos
df.isna().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  1
ocorrencia_aerodromo        2349
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [11]:
# Exibindo linha com valores null, na coluna ocorrencia_uf
filtro = df.ocorrencia_uf.isnull()
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2227,49474,49474,ACIDENTE,ÁGUAS INTERNACIONAIS,NaN,NaN,2013-09-02,02:54:00,0


In [12]:
# Exibindo linha com valores null, na coluna ocorrencia_aerodromo
filtro = df.ocorrencia_aerodromo.isnull()
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
5,39807,39807,INCIDENTE,SALVADOR,BA,NaN,2010-01-06,17:53:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0
...,...,...,...,...,...,...,...,...,...
5741,79754,79754,INCIDENTE GRAVE,NOVA MARINGÁ,MT,NaN,2020-12-28,13:00:00,0
5744,79753,79753,INCIDENTE GRAVE,GOIATUBA,GO,NaN,2020-12-29,12:00:00,0
5745,79755,79755,ACIDENTE,MATO RICO,PR,NaN,2020-12-29,10:30:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,NaN,2020-12-30,18:30:00,0


In [13]:
# Exibindo linha com valores null, na coluna ocorrencia_hora
filtro = df.ocorrencia_hora.isnull()
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
4100,78823,78823,INCIDENTE,CORUMBÁ,MS,NaN,2017-07-09,NaN,0


In [14]:
# ocorrencia com mais de 10 recomendações
filtro = df.total_recomendacoes > 10
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
63,39992,39992,ACIDENTE,SÃO PAULO,SP,SBMT,2010-02-17,20:07:00,11
326,42250,42250,ACIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-08-12,12:26:00,19
530,43471,43471,ACIDENTE,SÃO PAULO,SP,NaN,2010-12-14,20:15:00,11
727,44377,44377,ACIDENTE,ITÁPOLIS,SP,SDIO,2011-04-09,14:50:00,11
872,44796,44796,ACIDENTE,RECIFE,PE,SBRF,2011-07-13,09:54:00,23
884,44888,44888,INCIDENTE GRAVE,SÃO JOSÉ DOS PINHAIS,PR,SBCT,2011-07-15,17:05:00,11
1062,45554,45554,INCIDENTE GRAVE,UBERLÂNDIA,MG,SBUL,2011-11-15,02:15:00,20
1972,47938,47938,INCIDENTE,BRASÍLIA,DF,SBBR,2013-04-13,18:00:00,11
2795,52265,52265,ACIDENTE,SANTOS,SP,NaN,2014-08-13,13:03:00,13
3930,66432,66432,INCIDENTE GRAVE,VITÓRIA,ES,NaN,2017-02-21,11:47:00,12


In [15]:
# cidade com ocorrencias com mais de 10 recomendações
filtro = df.total_recomendacoes > 10
df.loc[filtro, ['ocorrencia_cidade', 'total_recomendacoes']]

,ocorrencia_cidade,total_recomendacoes
63,SÃO PAULO,11
326,RIO DE JANEIRO,19
530,SÃO PAULO,11
727,ITÁPOLIS,11
872,RECIFE,23
884,SÃO JOSÉ DOS PINHAIS,11
1062,UBERLÂNDIA,20
1972,BRASÍLIA,11
2795,SANTOS,13
3930,VITÓRIA,12


In [16]:
# cidade com ocorrencias do tipo: INCIDENTE GRAVE
filtro = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
df.loc[filtro, ['ocorrencia_cidade', 'ocorrencia_classificacao','total_recomendacoes']]

,ocorrencia_cidade,ocorrencia_classificacao,total_recomendacoes
7,CANUTAMA,INCIDENTE GRAVE,3
8,CASCAVEL,INCIDENTE GRAVE,2
9,PARÁ DE MINAS,INCIDENTE GRAVE,0
29,CURITIBA,INCIDENTE GRAVE,2
35,PALMAS,INCIDENTE GRAVE,0
...,...,...,...
5741,NOVA MARINGÁ,INCIDENTE GRAVE,0
5744,GOIATUBA,INCIDENTE GRAVE,0
5746,MANOEL URBANO,INCIDENTE GRAVE,0
5748,LAGOA DA CONFUSÃO,INCIDENTE GRAVE,0


In [17]:
# Filtrando ocorrencias do tipo: INCIDENTE GRAVE e o estado igual a SP
# Usando Condição: AND
filtro = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
136,40270,40270,INCIDENTE GRAVE,PIRASSUNUNGA,SP,SDPY,2010-04-15,20:15:00,0
171,40620,40620,INCIDENTE GRAVE,GUARULHOS,SP,SBGR,2010-05-06,15:50:00,0
219,41411,41411,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2010-06-08,19:30:00,2
297,42289,42289,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2010-07-29,20:06:00,0
314,42323,42323,INCIDENTE GRAVE,SÃO JOSÉ DO RIO PRETO,SP,SBSR,2010-08-05,12:01:00,0
...,...,...,...,...,...,...,...,...,...
5591,79540,79540,INCIDENTE GRAVE,SÃO PAULO,SP,NaN,2020-09-12,13:10:00,0
5610,79566,79566,INCIDENTE GRAVE,REGENTE FEIJÓ,SP,SDYJ,2020-09-27,19:33:00,0
5704,79739,79739,INCIDENTE GRAVE,SÃO PAULO,SP,SBSP,2020-12-03,14:42:00,0
5705,79705,79705,INCIDENTE GRAVE,SOROCABA,SP,SDCO,2020-12-04,15:30:00,0


In [18]:
# Filtrando ocorrencias do tipo: INCIDENTE GRAVE ou estado igual a SP
# Usando Condição: OR
filtro = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro | filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0
11,40069,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0
...,...,...,...,...,...,...,...,...,...
5744,79753,79753,INCIDENTE GRAVE,GOIATUBA,GO,NaN,2020-12-29,12:00:00,0
5746,79769,79769,INCIDENTE GRAVE,MANOEL URBANO,AC,SIMB,2020-12-29,18:30:00,0
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,NaN,2020-12-30,18:30:00,0


In [19]:
# Filtrando ocorrencias cuja classificação: 'INCIDENTE GRAVE' OR 'INCIDENTE' AND Estado igual a SP
# Usando Condição: OR e AND
filtro1 = df.ocorrencia_classificacao.isin(['INCIDENTE GRAVE','INCIDENTE'])
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0
18,39828,39828,INCIDENTE,SANTOS,SP,NaN,2010-01-15,17:45:00,0
26,39847,39847,INCIDENTE,SOROCABA,SP,NaN,2010-01-20,13:10:00,0
27,39768,39768,INCIDENTE,CAMPINAS,SP,NaN,2010-01-21,20:45:00,0
33,39848,39848,INCIDENTE,GUARULHOS,SP,SBGR,2010-01-25,17:05:00,0
...,...,...,...,...,...,...,...,...,...
5704,79739,79739,INCIDENTE GRAVE,SÃO PAULO,SP,SBSP,2020-12-03,14:42:00,0
5705,79705,79705,INCIDENTE GRAVE,SOROCABA,SP,SDCO,2020-12-04,15:30:00,0
5715,79718,79718,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2020-12-11,13:50:00,0
5742,79800,79800,INCIDENTE,SÃO PAULO,SP,SBMT,2020-12-28,10:15:00,0


In [20]:
# Ocorrencias cuja cidade começa com letra "C"
filtro1 = df.ocorrencia_cidade.str[0] == 'C' # startswith("C")
df.loc[filtro1]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
20,40310,40310,INCIDENTE,CRUZEIRO DO SUL,AC,SBCZ,2010-01-18,19:32:00,0
...,...,...,...,...,...,...,...,...,...
5661,79649,79649,INCIDENTE,CAMPINAS,SP,SBKP,2020-11-04,22:39:00,0
5663,79652,79652,INCIDENTE,CONFINS,MG,SBCF,2020-11-06,14:20:00,0
5680,79683,79683,INCIDENTE,CRUZEIRO DO SUL,AC,SBCZ,2020-11-16,22:00:00,0
5696,79692,79692,ACIDENTE,CATANDUVA,SP,SDCD,2020-11-28,12:50:00,0


In [21]:
# Ocorrencias cuja cidade termina com letra "A"
filtro1 = df.ocorrencia_cidade.str[-1] == 'A' # endswith("A")
df.loc[filtro1]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
12,40414,40414,INCIDENTE,BRASÍLIA,DF,SBBR,2010-01-10,03:00:00,0
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
25,39772,39772,INCIDENTE,UBERLÂNDIA,MG,NaN,2010-01-20,18:58:00,0
26,39847,39847,INCIDENTE,SOROCABA,SP,NaN,2010-01-20,13:10:00,0
...,...,...,...,...,...,...,...,...,...
5738,79787,79787,INCIDENTE,FORTALEZA,CE,SBFZ,2020-12-23,18:30:00,0
5739,79749,79749,ACIDENTE,MARITUBA,PA,NaN,2020-12-26,12:05:00,0
5743,79824,79824,ACIDENTE,RIO PARANAÍBA,MG,SNRP,2020-12-28,17:00:00,0
5744,79753,79753,INCIDENTE GRAVE,GOIATUBA,GO,NaN,2020-12-29,12:00:00,0


In [22]:
# Ocorrencias cuja cidade termina com os caracteres "US"
filtro1 = df.ocorrencia_cidade.str[-2:] == 'US'
df.loc[filtro1]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
78,39916,39916,INCIDENTE GRAVE,MANAUS,AM,SBEG,2010-03-07,19:35:00,2
100,40170,40170,INCIDENTE,MANAUS,AM,SBEG,2010-03-20,11:00:00,0
163,40575,40575,INCIDENTE,MANAUS,AM,SBEG,2010-05-01,16:15:00,0
174,40791,40791,INCIDENTE,MANAUS,AM,NaN,2010-05-07,10:28:00,0
180,40834,40834,INCIDENTE,ILHÉUS,BA,NaN,2010-05-10,23:55:00,0
...,...,...,...,...,...,...,...,...,...
5606,79591,79591,INCIDENTE,MANAUS,AM,SBEG,2020-09-20,18:50:00,0
5691,79709,79709,INCIDENTE,MANAUS,AM,SBEG,2020-11-24,01:00:00,0
5707,79712,79712,INCIDENTE GRAVE,BOM JESUS,PI,NaN,2020-12-06,11:00:00,0
5730,79785,79785,INCIDENTE,MANAUS,AM,SBEG,2020-12-19,12:37:00,0


In [23]:
# Ocorrencias cuja cidade contem os caracteres "US"
filtro1 = df.ocorrencia_cidade.str.contains('US')
df.loc[filtro1]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
78,39916,39916,INCIDENTE GRAVE,MANAUS,AM,SBEG,2010-03-07,19:35:00,2
100,40170,40170,INCIDENTE,MANAUS,AM,SBEG,2010-03-20,11:00:00,0
163,40575,40575,INCIDENTE,MANAUS,AM,SBEG,2010-05-01,16:15:00,0
174,40791,40791,INCIDENTE,MANAUS,AM,NaN,2010-05-07,10:28:00,0
180,40834,40834,INCIDENTE,ILHÉUS,BA,NaN,2010-05-10,23:55:00,0
...,...,...,...,...,...,...,...,...,...
5691,79709,79709,INCIDENTE,MANAUS,AM,SBEG,2020-11-24,01:00:00,0
5707,79712,79712,INCIDENTE GRAVE,BOM JESUS,PI,NaN,2020-12-06,11:00:00,0
5730,79785,79785,INCIDENTE,MANAUS,AM,SBEG,2020-12-19,12:37:00,0
5731,79746,79746,INCIDENTE,MANAUS,AM,SWFN,2020-12-20,20:40:00,0


In [24]:
# Ocorrencias cuja cidade contem os caracteres "MA" ou "US"
filtro1 = df.ocorrencia_cidade.str.contains('MA|US')
df.loc[filtro1]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
13,39507,39507,ACIDENTE,PRIMAVERA DO LESTE,MT,NaN,2010-01-11,17:00:00,2
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
35,39487,39487,INCIDENTE GRAVE,PALMAS,TO,SBPJ,2010-01-26,16:50:00,0
78,39916,39916,INCIDENTE GRAVE,MANAUS,AM,SBEG,2010-03-07,19:35:00,2
...,...,...,...,...,...,...,...,...,...
5739,79749,79749,ACIDENTE,MARITUBA,PA,NaN,2020-12-26,12:05:00,0
5741,79754,79754,INCIDENTE GRAVE,NOVA MARINGÁ,MT,NaN,2020-12-28,13:00:00,0
5745,79755,79755,ACIDENTE,MATO RICO,PR,NaN,2020-12-29,10:30:00,0
5746,79769,79769,INCIDENTE GRAVE,MANOEL URBANO,AC,SIMB,2020-12-29,18:30:00,0


In [25]:
# Ocorrencias do ano de 2015
filtro1 = df.ocorrencia_dia.dt.year == 2015
df.loc[filtro1]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2996,52992,52992,INCIDENTE GRAVE,SALVADOR,BA,SBSV,2015-01-01,11:40:00,3
2997,52979,52979,ACIDENTE,IVINHEMA,MS,NaN,2015-01-02,14:50:00,0
2998,53073,53073,INCIDENTE,TEFÉ,AM,SBTF,2015-01-02,16:28:00,0
2999,53074,53074,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-01-02,20:34:00,0
3000,52976,52976,ACIDENTE,TOLEDO,PR,SBTD,2015-01-04,22:04:00,6
...,...,...,...,...,...,...,...,...,...
3462,60632,60632,INCIDENTE GRAVE,ITABERÁ,SP,NaN,2015-12-24,14:00:00,0
3463,60600,60600,INCIDENTE,GUARULHOS,SP,SBGR,2015-12-25,19:00:00,0
3464,60642,60642,INCIDENTE,SÃO FRANCISCO DO SUL,SC,SSSS,2015-12-26,16:00:00,0
3465,60631,60631,ACIDENTE,MAÇAMBARÁ,RS,NaN,2015-12-28,19:00:00,2


In [26]:
# Ocorrencias do ano de 2015 e mês 12
## filtro1 = (df.ocorrencia_dia.dt.year == 2015) & (df.ocorrencia_dia.dt.month == 12)
filtro1 = df.ocorrencia_dia.dt.year == 2015
filtro2 = df.ocorrencia_dia.dt.month == 12
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3428,53573,53573,INCIDENTE,GUARULHOS,SP,SBGR,2015-12-01,02:48:00,0
3429,60601,60601,INCIDENTE,PALMAS,TO,SBPJ,2015-12-01,16:05:00,0
3430,53634,53634,INCIDENTE,PALMAS,TO,SBPJ,2015-12-02,17:45:00,0
3431,53636,53636,INCIDENTE,JUNDIAÍ,SP,SBJD,2015-12-02,17:42:00,0
3432,53575,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0
3433,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0
3434,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3
3435,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1
3436,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1
3437,53629,53629,ACIDENTE,JALES,SP,SDJL,2015-12-08,10:20:00,0


In [27]:
# Ocorrencias do ano de 2015 e mês 12 e os dias entre 3 e 8
filtro_ano = df.ocorrencia_dia.dt.year == 2015
filtro_mes = df.ocorrencia_dia.dt.month == 12
filtro_dia = (df.ocorrencia_dia.dt.day >= 3) & (df.ocorrencia_dia.dt.day <= 8)
df.loc[filtro_ano & filtro_mes & filtro_dia]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
3432,53575,53575,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,SBFS,2015-12-03,10:50:00,0
3433,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0
3434,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3
3435,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1
3436,53628,53628,ACIDENTE,AGUAÍ,SP,NaN,2015-12-08,14:30:00,1
3437,53629,53629,ACIDENTE,JALES,SP,SDJL,2015-12-08,10:20:00,0
3438,53631,53631,INCIDENTE,CAMPINAS,SP,SBKP,2015-12-08,16:19:00,0
3439,60636,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0


In [28]:
# Unificando DATE com STRING
# Utilizamos "pd.to_datetime" para criar um datetime
df['ocorrencia_dia_hora'] = pd.to_datetime(df.ocorrencia_dia.astype(str) + ' ' + df.ocorrencia_hora)

In [29]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
ocorrencia_dia_hora         datetime64[ns]
dtype: object

In [30]:
df.head()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0,2010-01-03 12:00:00
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0,2010-01-03 11:05:00
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0,2010-01-03 03:00:00
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0,2010-01-04 17:30:00
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0,2010-01-05 19:25:00


In [32]:
# Ocorrencias do ano de 2015 e mês 12 e os dias entre 3 e 8
filtro1 = df.ocorrencia_dia_hora >= '2015-12-03 11:00:00'
filtro2 = df.ocorrencia_dia_hora <= '2015-12-08 13:00:00'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
3433,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0,2015-12-03 16:47:00
3434,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3,2015-12-06 13:10:00
3435,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1,2015-12-06 15:00:00
3437,53629,53629,ACIDENTE,JALES,SP,SDJL,2015-12-08,10:20:00,0,2015-12-08 10:20:00
3439,60636,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0,2015-12-08 13:00:00


In [33]:
# ocorrencias do anor de 2015 e mês 03
filtro1 = df.ocorrencia_dia.dt.year == 2015
filtro2 = df.ocorrencia_dia.dt.month == 3
df201503 = df.loc[filtro1 & filtro2]

In [35]:
df201503.head(5)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
3086,53183,53183,ACIDENTE,AMAPORÃ,PR,NaN,2015-03-02,23:00:00,0,2015-03-02 23:00:00
3087,53120,53120,ACIDENTE,CHAVES,PA,SNXW,2015-03-04,13:30:00,0,2015-03-04 13:30:00
3088,53109,53109,ACIDENTE,CAMPO GRANDE,MS,SSIE,2015-03-05,13:50:00,0,2015-03-05 13:50:00
3089,53112,53112,ACIDENTE,MOGI GUAÇU,SP,NaN,2015-03-06,21:00:00,0,2015-03-06 21:00:00
3090,53152,53152,INCIDENTE GRAVE,RIO DE JANEIRO,RJ,SBRJ,2015-03-10,11:30:00,2,2015-03-10 11:30:00


In [40]:
# Agrupando e contando dados
df201503.groupby(['ocorrencia_classificacao']).codigo_ocorrencia.count()

ocorrencia_classificacao
ACIDENTE           15
INCIDENTE          17
INCIDENTE GRAVE     5
Name: codigo_ocorrencia, dtype: int64

In [41]:
# Agrupando e contando dados
df201503.groupby(['ocorrencia_classificacao']).ocorrencia_aerodromo.count()

ocorrencia_classificacao
ACIDENTE            5
INCIDENTE          14
INCIDENTE GRAVE     2
Name: ocorrencia_aerodromo, dtype: int64

In [42]:
# Agrupando e exbibindo o tamanho, que não é alterado por valores nulos
df201503.groupby(['ocorrencia_classificacao']).size()

ocorrencia_classificacao
ACIDENTE           15
INCIDENTE          17
INCIDENTE GRAVE     5
dtype: int64

In [43]:
filtro1 = df.ocorrencia_dia.dt.year == 2010
filtro2 = df.ocorrencia_uf.isin(['SP', 'MG', 'ES', 'RJ'])
dfsudeste2010 = df.loc[filtro1 & filtro2]
dfsudeste2010

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0,2010-01-03 12:00:00
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0,2010-01-03 03:00:00
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0,2010-01-10 20:00:00
11,40069,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8,2010-01-10 14:50:00
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0,2010-01-15 15:00:00
...,...,...,...,...,...,...,...,...,...,...
541,43722,43722,ACIDENTE,CAJOBI,SP,NaN,2010-12-23,11:00:00,0,2010-12-23 11:00:00
544,43548,43548,INCIDENTE,SÃO PAULO,SP,SBSP,2010-12-26,22:10:00,0,2010-12-26 22:10:00
548,43724,43724,ACIDENTE,COSMÓPOLIS,SP,NaN,2010-12-29,12:00:00,0,2010-12-29 12:00:00
550,43730,43730,INCIDENTE,VITÓRIA,ES,NaN,2010-12-30,18:10:00,0,2010-12-30 18:10:00


In [44]:
dfsudeste2010.groupby(['ocorrencia_classificacao']).size()

ocorrencia_classificacao
ACIDENTE            40
INCIDENTE          170
INCIDENTE GRAVE     18
dtype: int64

In [45]:
dfsudeste2010.count()

codigo_ocorrencia           228
codigo_ocorrencia2          228
ocorrencia_classificacao    228
ocorrencia_cidade           228
ocorrencia_uf               228
ocorrencia_aerodromo        127
ocorrencia_dia              228
ocorrencia_hora             228
total_recomendacoes         228
ocorrencia_dia_hora         228
dtype: int64

In [46]:
dfsudeste2010.groupby(['ocorrencia_classificacao', 'ocorrencia_uf']).size()

ocorrencia_classificacao  ocorrencia_uf
ACIDENTE                  MG                7
                          RJ                9
                          SP               24
INCIDENTE                 ES                5
                          MG               39
                          RJ               52
                          SP               74
INCIDENTE GRAVE           MG                5
                          RJ                4
                          SP                9
dtype: int64

In [47]:
dfsudeste2010.groupby(['ocorrencia_cidade']).size().sort_values(ascending=False)

ocorrencia_cidade
RIO DE JANEIRO           47
SÃO PAULO                33
GUARULHOS                18
BELO HORIZONTE           17
CAMPINAS                 13
MACAÉ                    12
CONFINS                   7
UBERLÂNDIA                5
VITÓRIA                   4
JUIZ DE FORA              4
SÃO JOSÉ DO RIO PRETO     3
SÃO JOSÉ DOS CAMPOS       3
MARÍLIA                   3
ARARAQUARA                2
MONTES CLAROS             2
CAMPOS DOS GOYTACAZES     2
DIVINÓPOLIS               2
IPATINGA                  2
ITANHAÉM                  2
JUNDIAÍ                   2
POÇOS DE CALDAS           2
RIBEIRÃO PRETO            2
SOROCABA                  2
UBERABA                   2
BAURU                     2
AREALVA                   2
BOITUVA                   1
BOCAINA                   1
BOM JESUS DO AMPARO       1
BOM JESUS DO GALHO        1
BEBEDOURO                 1
BRAGANÇA PAULISTA         1
BROTAS                    1
CAJOBI                    1
ARRAIAL DO CABO           1
CA

In [50]:
filtro1 = dfsudeste2010.ocorrencia_cidade == 'RIO DE JANEIRO'
filtro2 = dfsudeste2010.total_recomendacoes > 0
dfsudeste2010.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
119,40190,40190,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2010-04-06,02:35:00,3,2010-04-06 02:35:00
274,41750,41750,ACIDENTE,RIO DE JANEIRO,RJ,SBJR,2010-07-12,19:53:00,3,2010-07-12 19:53:00
326,42250,42250,ACIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-08-12,12:26:00,19,2010-08-12 12:26:00


In [52]:
# Soma do total de recomendações por cidade
dfsudeste2010.groupby(['ocorrencia_cidade']).total_recomendacoes.sum()

ocorrencia_cidade
AMERICANA                 0
ARARAQUARA                0
ARAÇATUBA                 0
AREALVA                   0
ARRAIAL DO CABO           0
BAURU                     0
BEBEDOURO                 0
BELO HORIZONTE            2
BOCAINA                   0
BOITUVA                   1
BOM JESUS DO AMPARO       0
BOM JESUS DO GALHO        1
BRAGANÇA PAULISTA         9
BROTAS                    0
CAJOBI                    0
CAMANDUCAIA               9
CAMPINAS                  0
CAMPOS DOS GOYTACAZES     0
CASIMIRO DE ABREU         0
CONFINS                   0
COROMANDEL                0
COSMÓPOLIS                0
DIAMANTINA                0
DIVINÓPOLIS               0
GUARULHOS                 0
IPATINGA                  0
IPERÓ                     3
ITANHAÉM                  1
ITÁPOLIS                  0
JUIZ DE FORA              0
JUNDIAÍ                   7
LINHARES                  0
MACAÉ                     0
MANGARATIBA               0
MARÍLIA                   0
MO